<a href="https://colab.research.google.com/github/Alihassan7726/Text-classification-using-ULMFiT/blob/main/BBC_News_classification_using_ULMFiT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of Contents
1. Importing Modules 
2. Getting the Data
3. Cleaning Data
4. Splitting Data into train and val sets
5. Creating Data for forward language model
6. Creating Data for backward language model
7. Fine-tuning forward language model 
8. Fine-tuning backward language model
9. Creating Data for forward Classifier
10. Creating Data for backward Classifier
11. Training forward Classifier 
12. Training backward Classifier
13. Ensembling both predictions


## 1. Importing Modules 


In [ ]:
# For installation
pip install fastai

In [ ]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os
pd.set_option('display.width',1000)
pd.set_option('max_colwidth', 1000) 
import pandas as pd

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stpwrds = set(stopwords.words('english'))
stpwrds.discard('not')
stpwrds.discard('no')
import os
import re
from nltk.stem import WordNetLemmatizer as wnl
import sklearn


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


## 2. Getting the Data

We'll be using BBC news headlines DataSet . It can be downloaded easily using below link . 

In [ ]:
# Data is fetched and stored in tmp folder with name bbc-text.csv
!wget --no-check-certificate \
    https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv \
    -O /tmp/bbc-text.csv

--2021-06-08 05:08:45--  https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 2607:f8b0:4023:c0b::80, 2607:f8b0:4023:c03::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [text/csv]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  --.-KB/s    in 0.02s   

2021-06-08 05:08:45 (257 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [ ]:
df = pd.read_csv('/tmp/bbc-text.csv' , delimiter = ',')
df.head()

,category,text
0,tech,tv future in the hands of viewers with home theatre systems plasma high-definition tvs and digital video recorders moving into the living room the way people watch tv will be radically different in five years time. that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend programmes and other content will be delivered to viewers via home networks through cable satellite telecoms companies and broadband service providers to front rooms and portable devices. one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes like the us s tivo and the uk s sky+ system allow people to record store play pause and forward wind tv programmes when they want. essentially the technology allows for much more personalised tv. they are also being built-in to h...
1,business,worldcom boss left books alone former worldcom boss bernie ebbers who is accused of overseeing an $11bn (£5.8bn) fraud never made accounting decisions a witness has told jurors. david myers made the comments under questioning by defence lawyers who have been arguing that mr ebbers was not responsible for worldcom s problems. the phone company collapsed in 2002 and prosecutors claim that losses were hidden to protect the firm s shares. mr myers has already pleaded guilty to fraud and is assisting prosecutors. on monday defence lawyer reid weingarten tried to distance his client from the allegations. during cross examination he asked mr myers if he ever knew mr ebbers make an accounting decision . not that i am aware of mr myers replied. did you ever know mr ebbers to make an accounting entry into worldcom books mr weingarten pressed. no replied the witness. mr myers has admitted that he ordered false accounting entries at the request of former worldcom chief fina...
2,sport,tigers wary of farrell gamble leicester say they will not be rushed into making a bid for andy farrell should the great britain rugby league captain decide to switch codes. we and anybody else involved in the process are still some way away from going to the next stage tigers boss john wells told bbc radio leicester. at the moment there are still a lot of unknowns about andy farrell not least his medical situation. whoever does take him on is going to take a big big gamble. farrell who has had persistent knee problems had an operation on his knee five weeks ago and is expected to be out for another three months. leicester and saracens are believed to head the list of rugby union clubs interested in signing farrell if he decides to move to the 15-man game. if he does move across to union wells believes he would better off playing in the backs at least initially. i m sure he could make the step between league and union by being involved in the centre said wells. ...
3,sport,yeading face newcastle in fa cup premiership side newcastle united face a trip to ryman premier league leaders yeading in the fa cup third round. the game - arguably the highlight of the draw - is a potential money-spinner for non-league yeading who beat slough in the second round. conference side exeter city who knocked out doncaster on saturday will travel to old trafford to meet holders manchester united in january. arsenal were drawn at home to stoke and chelsea will play host to scunthorpe. the only other non-league side in the draw are hinckley united who held brentford to a goalless draw on sunday. they will meet league one leaders luton if they win their replay against martin allen s team at griffin park. a number of premiership teams face difficult away games against championship sides on the weekend of 8/9 january. third-placed everton visit plymouth liverpool travel to burnley crystal palace go to sunderland fulham face carling cup semi-finalists watford bolton...
4,entertainment,ocea

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   text      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [ ]:
df['category'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64

In [ ]:
# For simplicity we'll be taking only 2 classes for this task
df = df[df['category'].isin(['tech','business'])]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 911 entries, 0 to 2220
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  911 non-null    object
 1   text      911 non-null    object
dtypes: object(2)
memory usage: 21.4+ KB


## 3. Cleaning Data


In [ ]:
# A simple function for data cleaning

def clean_text(text):  
    lower = text.lower()
    words = re.sub(r"(@[A-Za-z]+)|([^A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", lower )  
    #pattren = r"[^\w\s\d]"
    #words = re.sub(pattren, "", lower)
    words2 = words.split()
    #print(words2)
    final_words =  [wnl().lemmatize(word , pos = 'v') for word in words2 if word not in stopwords.words('english')]
    final_words = ' '.join(final_words)
    return(final_words)
    
df['Cleaned-text'] = df['text'].apply(clean_text)
df.head()

,category,text,Cleaned-text
0,tech,tv future in the hands of viewers with home theatre systems plasma high-definition tvs and digital video recorders moving into the living room the way people watch tv will be radically different in five years time. that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend programmes and other content will be delivered to viewers via home networks through cable satellite telecoms companies and broadband service providers to front rooms and portable devices. one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes like the us s tivo and the uk s sky+ system allow people to record store play pause and forward wind tv programmes when they want. essentially the technology allows for much more personalised tv. they are also being built-in to h...,tv future hand viewers home theatre systems plasma highdefinition tvs digital video recorders move live room way people watch tv radically different five years time accord expert panel gather annual consumer electronics show las vegas discuss new technologies impact one favourite pastimes us lead trend program content deliver viewers via home network cable satellite telecoms company broadband service providers front room portable devices one talkedabout technologies ces digital personal video recorders dvr pvr settop box like us tivo uk sky system allow people record store play pause forward wind tv program want essentially technology allow much personalise tv also builtin highdefinition tv set big business japan us slower take europe lack highdefinition program people forward wind advert also forget abide network channel schedule put together alacarte entertainment us network cable satellite company worry mean term advertise revenues well brand identity viewer loyalty channel alth...
1,business,worldcom boss left books alone former worldcom boss bernie ebbers who is accused of overseeing an $11bn (£5.8bn) fraud never made accounting decisions a witness has told jurors. david myers made the comments under questioning by defence lawyers who have been arguing that mr ebbers was not responsible for worldcom s problems. the phone company collapsed in 2002 and prosecutors claim that losses were hidden to protect the firm s shares. mr myers has already pleaded guilty to fraud and is assisting prosecutors. on monday defence lawyer reid weingarten tried to distance his client from the allegations. during cross examination he asked mr myers if he ever knew mr ebbers make an accounting decision . not that i am aware of mr myers replied. did you ever know mr ebbers to make an accounting entry into worldcom books mr weingarten pressed. no replied the witness. mr myers has admitted that he ordered false accounting entries at the request of former worldcom chief fina...,worldcom boss leave book alone former worldcom boss bernie ebbers accuse oversee bn bn fraud never make account decisions witness tell jurors david myers make comment question defence lawyers argue mr ebbers responsible worldcom problems phone company collapse prosecutors claim losses hide protect firm share mr myers already plead guilty fraud assist prosecutors monday defence lawyer reid weingarten try distance client allegations cross examination ask mr myers ever know mr ebbers make account decision aware mr myers reply ever know mr ebbers make account entry worldcom book mr weingarten press reply witness mr myers admit order false account entries request former worldcom chief financial officer scott sullivan defence lawyers try paint mr sullivan admit fraud testify later trial mastermind behind worldcom account house card mr ebbers team meanwhile look portray affable boss admission pe graduate economist whatever abilities mr ebbers transform worldcom relative unknown bn tel

In [ ]:
df.drop(columns=['text'],inplace=True)
df.head(2)

,category,Cleaned-text
0,tech,tv future hand viewers home theatre systems plasma highdefinition tvs digital video recorders move live room way people watch tv radically different five years time accord expert panel gather annual consumer electronics show las vegas discuss new technologies impact one favourite pastimes us lead trend program content deliver viewers via home network cable satellite telecoms company broadband service providers front room portable devices one talkedabout technologies ces digital personal video recorders dvr pvr settop box like us tivo uk sky system allow people record store play pause forward wind tv program want essentially technology allow much personalise tv also builtin highdefinition tv set big business japan us slower take europe lack highdefinition program people forward wind advert also forget abide network channel schedule put together alacarte entertainment us network cable satellite company worry mean term advertise revenues well brand identity viewer loyalty channel alth...
1,business,worldcom boss leave book alone former worldcom boss bernie ebbers accuse oversee bn bn fraud never make account decisions witness tell jurors david myers make comment question defence lawyers argue mr ebbers responsible worldcom problems phone company collapse prosecutors claim losses hide protect firm share mr myers already plead guilty fraud assist prosecutors monday defence lawyer reid weingarten try distance client allegations cross examination ask mr myers ever know mr ebbers make account decision aware mr myers reply ever know mr ebbers make account entry worldcom book mr weingarten press reply witness mr myers admit order false account entries request former worldcom chief financial officer scott sullivan defence lawyers try paint mr sullivan admit fraud testify later trial mastermind behind worldcom account house card mr ebbers team meanwhile look portray affable boss admission pe graduate economist whatever abilities mr ebbers transform worldcom relative unknown bn telecom...


## 4. Splitting Data into train and val sets

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df, stratify = df['category'], test_size = 0.1, random_state = 12)

print(df_train.shape)
print(df_val.shape)

(819, 2)
(92, 2)


In [ ]:
del(df)

## 5. Creating Data for forward language model


In [ ]:
# Batch-size and BPTT
bs,bptt=32,40


In [ ]:
# Language model data
# This part may take some time
data_lm = TextLMDataBunch.from_df(train_df = df_train, valid_df = df_val, path = "" )

data_lm.save('data_lm.pkl')


/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


In [ ]:
del(data_lm)
path = "/content/"
data_lm = load_data(path , 'data_lm.pkl', bs=bs, bptt=bptt)


In [ ]:
data_lm.show_batch()


idx,text
0,highest rank executive plead guilty tuesday former marsh senior vice president joshua bewlay admit one felony count scheme defraud face four years prison marsh spokeswoman say mr bewlay longer company mr spitzer investigation us insurance industry look whether company rig
1,cash allow cheap plenty room improve could end make expensive long run good basic xxunk pretty monitor rather xxunk display upgrade offer xxunk basic model xxunk might find want xxunk change quite quickly nick ross deputy labs editor pc pro
2,pop people want portability say peertopeer portability cory doctorow european coordinator electronic frontier foundation campaign consumers many cyberrights issue express doubt marlin system would achieve aim one systems ever prevent piracy illegal copy say say many firm readily admit drm
3,new seafarers identity document incorporate barcode represent unique feature holder fingerprint card due issue february next year line revise un convention seafarers identity document june test currently way caribbean design ensure new card machine readers produce different company different countries
4,remain xxunk malaysia thirdbiggest lender already scout foreign partner new islamic bank unit firm tell reuters move put malaysia ahead deadline open sector country deal join world trade organisation set year deadline liberalisation islamic bank also tuesday central bank release


## 6. Creating Data for backward language model

This methods reads the data from backwards just like reverse part of normal Bi-directional LSTM . It can be easily implemented by loading the same "data_lm" object with backwards = True argument that reads the data from reverse side .  

In [ ]:
data_bwd = load_data(path, 'data_lm.pkl', bs=bs, bptt=bptt, backwards=True)
data_bwd.show_batch()


idx,text
0,work firm game video say bolz mr movies music game video combine would xxunk xxunk similar show award international plan ea say also bolz mr say show play world gamers mechanism bedroom go kitchen go tell family control could idea
1,eight attention commission bring aid state possible matter swiftly complete analysis like would aid state contain plan confirm aspects certain verify xxunk mr say plan industrial serious present authorities italian possible swiftly carry want say barrot jacques commissioner transport however
2,hard xxunk ripe audiences xxunk interest capture others sims game ability phenomenal believe say xxunk inherently play learn xxunk false believe break would goal one game embrace teachers language particularly educationalists believe purushotma ravi game computer fear instead xxunk really
3,rat interest keep decision change rate vote bank xxbos live fan gadget problem kind capacities hold data vast feature number enormous gadgets array bewilder confront choices tough series face consumers downside want way almost communicate anything almost able gadget every
4,correction rule shortlived may deficits budget trade us massive worry currencies major lower push late spend dollar respite say economists february early meet next rat boost favour odds rat interest set xxunk committee market open federal say market financial xxunk


## 7. Fine-tuning forward language model

The vocabulary of the two datasets ( BBC and Wikipedia ) are slightly different, so when loading the weights, we take care to put the embedding weights at the right place, and we randomly initiliaze the embeddings for words in the BBC vocabulary that weren't in the wikitext-103 vocabulary of our pretrained model. 

This is all done in just 1st line of code below

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM , drop_mult=1.0)
learn = learn.to_fp16(clip=0.1) # Speeds up training on a GPU

In [ ]:
lr = 1e-2
#lr *= bs/48
lr /= 8
lr

0.00125

In [ ]:
# The Learner object we get is frozen by default, which means we only train the embeddings at first
learn.fit_one_cycle(1, lr, moms=(0.8,0.7), wd=0.1)


epoch,train_loss,valid_loss,accuracy,time
0,7.713527,7.048201,0.060547,00:04


In [ ]:
print(2e-3)
print(lr/5)

0.002
0.00025


In [ ]:
# Then we unfreeze the model and fine-tune the whole thing for 10 epochs 
learn.unfreeze()
learn.fit_one_cycle(18, lr/5, moms=(0.8,0.7), wd=0.1)


In [ ]:
# Once done we will save our encoder
learn.save_encoder('fwd_enc')


## 8. Fine-tuning backward language model

We can't directly train a bidirectional RNN for language modeling, but we can always ensemble a forward and backward model. fastai provides a pretrained forward and backawrd model, so we can repeat the previous step to fine-tune the pretrained backward model. The command language_model_learner checks the data object you pass to automatically decide if it should use the pretrained forward or backward model.

In [ ]:
learn = language_model_learner(data_bwd, AWD_LSTM)
learn = learn.to_fp16(clip=0.1)

In [ ]:
# The rest of process is same

lr = 0.00125
learn.fit_one_cycle(1, lr, moms=(0.8,0.7), wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,7.874811,7.089356,0.052197,00:04


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(15, lr/5, moms=(0.8,0.7), wd=0.1)


In [ ]:
learn.save_encoder('bwd_enc')


## 9. Creating Data for forward Classifier

The classifier is a model that is a bit heavier, so we have lower the batch size.


In [ ]:
bs = 32

In [ ]:
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_train, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=bs)

data_clas.save('data_clas.pkl')


/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
del(data_clas)
data_clas = load_data(path, 'data_clas.pkl', bs=bs)
data_clas.show_batch()


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


text,target
xxbos lose online game online role play game timeconsuming xxunk flight reality people take fantasy live seriously video game world warcraft hit shop europe last week fan write bbc website express delight offer warn addiction game like far costly time substance could impair keep track time write travis xxunk texas comment xxunk game good go get way could limit hours spend play write charles xxunk england strike worry tone massively,tech
xxbos apple laptop greatest gadget apple powerbook choose greatest gadget time us magazine mobile pc laptop choose one first lightweight portable computers help define layout future notebook pcs magazine compile alltime top list gadgets include sony walkman number three zenith remote control two gadgets need move part andor electronics warrant inclusion magazine staff compile list specify gadgets also need selfcontained apparatus use subset another device general include items potentially mobile,tech
xxbos call action internet scam phone company enough warn customers internet roguedialling scam accord premium phone line regulator icstis receive complaints recent months dialup internet connections divert premium rate number without users knowledge phone company refuse pay compensation say call must pay must warn people earlier possible fraud icstis say people use dialup connections affect scam without realise program download divert internet call via premium phone line victims often fail,tech
xxbos apple laptop greatest gadget apple powerbook choose greatest gadget time us magazine mobile pc laptop choose one first lightweight portable computers help define layout future notebook pcs magazine compile alltime top list gadgets include sony walkman number three zenith remote control two gadgets need move part andor electronics warrant inclusion magazine specify gadgets also need selfcontained apparatus use subset another device general include items potentially mobile say magazine end,tech
xxbos new console promise big problems make game future console require graphic artists money industry conference tell sony microsoft nintendo debut new console annual e game expo los angeles may socalled next generation machine faster current console capable display much xxunk visuals gamers make better immersive game prerecord video slot microsoft keynote address game developers conference hold last week san francisco xxunk director jam cameron reveal make game tandem next,tech


## 10. Creating Data for backward Classifier


In [ ]:
# Like before we will load Data for Backward Model
data_clas_bwd = load_data(path, 'data_clas.pkl', bs=bs, backwards=True)
data_clas_bwd.show_batch()


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


text,target
xxunk days xxunk xxunk xxrep xxunk game online ultima term half school kid hours meaningless absolutely universe online worry xxunk make trouble issue overcome swiftly world notice machine get need xxunk people worse get decline expect health persons damage time hours screen computer star proof medical xxunk unless children let know parent think nt do play card credit need game mmorpg xxunk adult end hours play gamers generation younger concern,tech
win deserve apple congratulations use cumbersome uncomfortable inaccurate invariably use today market notebook every trackpad useless largely way give trackball demise dishearten particularly pb like offer newest apple even notebooks modern wish live program taste give rubbish game tv plug k spectrum sinclair model nokia probably list high phone mobile put live earn world move never would beauty little without spectrum sinclair back would around info carry way practical first,tech
scam majority vast prevent date software firewall antivirus ensure responsibility take users home connection attempt sound hear modem connect nothing broadband upgrade already god thank scam target also back years write operator telecoms easier far limit would amount dispute least usage fraudulent limit number rate premium legitimate use could way lift provider contact would call rate premium say cap monthly automatic losses portion xxunk crime profit vat government supplier steal,tech
win deserve apple congratulations use cumbersome uncomfortable inaccurate invariably use today market notebook every trackpad useless largely way give trackball demise dishearten particularly pb like offer newest apple even notebooks modern wish live program taste give rubbish game tv plug k spectrum sinclair model nokia probably list high phone mobile put live earn world move never would beauty little without spectrum sinclair back would around info carry way practical first,tech
magazine develop editor xxunk xxunk console handheld psp upcoming sony quality comparable game run capable would phone mobile predict batter mr team development person mobile ea create game race speed need version first months next game mobile release plan ea franchise popular versions mobile release market dominate plan ea say admit batter mr business ea check time last phone game dismissive arts electronic recently publisher game biggest arts electronic division,tech


## 11. Training forward Classifier 

The classifier needs a little less dropout, so we pass drop_mult=0.5 to multiply all the dropouts by this amount (it's easier than adjusting all the five different values manually). We don't load the pretrained model, but instead our fine-tuned encoder from the previous section.

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False)
learn.load_encoder('fwd_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (819 items)
x: TextList
xxbos insurance boss plead guilty another three us insurance executives plead guilty fraud charge stem ongoing investigation industry xxunk two executives american international group aig one marsh mclennan latest investigation new york attorney general eliot spitzer obtain nine guilty pleas highest rank executive plead guilty tuesday former marsh senior vice president joshua bewlay admit one felony count scheme defraud face four years prison marsh spokeswoman say mr bewlay longer company mr spitzer investigation us insurance industry look whether company rig bid fix price last month marsh agree pay settle lawsuit file mr spitzer settlement neither admit deny allegations,xxbos long life promise laptop pcs scientists work ways ensure laptops stay power entire work day build batteries new chemical mix could boost power significantly say industry experts change include everything way chip laptops make trick reduc

Then we train the model using gradual unfreezing (partially training the model from everything but the classification head frozen to the whole model trianing by unfreezing one layer at a time) and differential learning rate (deeper layer gets a lower learning rate).

In [ ]:
lr = 2e-2
lr *= bs/48
lr

0.013333333333333332

In [ ]:
learn.fit_one_cycle(1, lr, moms=(0.8,0.7), wd=0.1)


epoch,train_loss,valid_loss,accuracy,time
0,0.247073,0.093189,0.967391,00:04


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
learn.freeze_to(-2)
lr /= 2
learn.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)


epoch,train_loss,valid_loss,accuracy,time
0,0.243828,0.075982,0.967391,00:05


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
slice(lr/(2.6**4),lr)

slice(0.00014588658193573985, 0.006666666666666666, None)

In [ ]:
import torch
torch.cuda.empty_cache() # To avoid CUDA GPU memory errors

In [ ]:
import gc
variables = gc.collect()
del variables


In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  222558 KB |    2822 MB |    4991 GB |    4991 GB |\n|       from large pool |  220842 KB |    2806 MB |    4847 GB |    4847 GB |\n|       from small pool |    1716 KB |      16 MB |     144 GB |     144 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |  222558 KB |    2822 MB |    4991 GB |    4991 GB |\n|       from large pool |  220842 KB |    2806 MB |

In [ ]:
learn.freeze_to(-3)
lr /= 2
learn.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.149340,0.060804,0.978261,00:08


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
learn.unfreeze()
lr /= 5
learn.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.098621,0.044541,0.978261,00:11
1,0.104822,0.048841,0.978261,00:11


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ar

In [ ]:
learn.save('fwd_clas')


## 12. Training backward Classifier



In [ ]:
learn_bwd = text_classifier_learner(data_clas_bwd, AWD_LSTM, drop_mult=0.5, pretrained=False)
learn_bwd.load_encoder('bwd_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (819 items)
x: TextList
xxbos insurance boss plead guilty another three us insurance executives plead guilty fraud charge stem ongoing investigation industry xxunk two executives american international group aig one marsh mclennan latest investigation new york attorney general eliot spitzer obtain nine guilty pleas highest rank executive plead guilty tuesday former marsh senior vice president joshua bewlay admit one felony count scheme defraud face four years prison marsh spokeswoman say mr bewlay longer company mr spitzer investigation us insurance industry look whether company rig bid fix price last month marsh agree pay settle lawsuit file mr spitzer settlement neither admit deny allegations,xxbos long life promise laptop pcs scientists work ways ensure laptops stay power entire work day build batteries new chemical mix could boost power significantly say industry experts change include everything way chip laptops make trick reduc

In [ ]:
lr = 2e-2
lr *= bs/48
lr
learn_bwd.fit_one_cycle(1, lr, moms=(0.8,0.7), wd=0.1)


epoch,train_loss,valid_loss,accuracy,time
0,0.301723,0.135776,0.989130,00:04


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
learn_bwd.freeze_to(-2)
lr /= 2
learn_bwd.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.214419,0.039729,1.000000,00:06


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
learn_bwd.freeze_to(-3)
lr /= 2
learn_bwd.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.114676,0.035128,0.989130,00:10


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
learn_bwd.unfreeze()
lr /= 5
learn_bwd.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.112508,0.026348,0.989130,00:13
1,0.111519,0.024389,0.989130,00:12


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ar

In [ ]:
learn_bwd.save('bwd_clas')


## 13. Ensembling both predictions

For our final results, we'll take the average of the predictions of the forward and the backward models. SInce the samples are sorted by text lengths for batching, we pass the argument ordered=True to get the predictions in the order of the texts.

In [ ]:
pred_fwd, targets_fwd = learn.get_preds(ordered=True)


In [ ]:
pred_bwd,targets_bwd = learn_bwd.get_preds(ordered=True)


In [ ]:
pred_fwd

tensor([[1.4341e-03, 9.9857e-01],
        [9.9909e-01, 9.0572e-04],
        [9.9669e-01, 3.3059e-03],
        [9.8903e-01, 1.0972e-02],
        [2.2430e-04, 9.9978e-01],
        [9.9830e-01, 1.7035e-03],
        [1.3542e-03, 9.9865e-01],
        [7.1861e-04, 9.9928e-01],
        [3.3345e-04, 9.9967e-01],
        [1.1111e-02, 9.8889e-01],
        [2.3549e-03, 9.9765e-01],
        [9.9788e-01, 2.1192e-03],
        [2.4670e-03, 9.9753e-01],
        [9.9488e-01, 5.1151e-03],
        [5.2103e-03, 9.9479e-01],
        [1.4343e-04, 9.9986e-01],
        [9.5807e-01, 4.1933e-02],
        [9.9510e-01, 4.8978e-03],
        [7.6834e-04, 9.9923e-01],
        [2.9621e-04, 9.9970e-01],
        [9.9555e-01, 4.4512e-03],
        [5.7628e-04, 9.9942e-01],
        [9.9980e-01, 2.0405e-04],
        [9.9829e-01, 1.7125e-03],
        [9.9698e-01, 3.0220e-03],
        [2.9042e-03, 9.9710e-01],
        [9.9549e-01, 4.5138e-03],
        [9.9532e-01, 4.6767e-03],
        [6.6835e-04, 9.9933e-01],
        [1.272

In [ ]:
9.9857e-01

0.99857

In [ ]:
final_pred = (pred_fwd+pred_bwd)/2
final_pred

tensor([[9.2034e-04, 9.9908e-01],
        [9.9607e-01, 3.9272e-03],
        [9.9743e-01, 2.5699e-03],
        [9.9101e-01, 8.9873e-03],
        [1.6438e-02, 9.8356e-01],
        [9.9876e-01, 1.2427e-03],
        [1.0829e-03, 9.9892e-01],
        [4.9118e-04, 9.9951e-01],
        [6.6969e-03, 9.9330e-01],
        [5.5375e-02, 9.4463e-01],
        [4.2907e-03, 9.9571e-01],
        [9.9726e-01, 2.7435e-03],
        [2.6566e-03, 9.9734e-01],
        [9.9391e-01, 6.0894e-03],
        [2.3101e-02, 9.7690e-01],
        [3.5853e-04, 9.9964e-01],
        [9.6624e-01, 3.3756e-02],
        [9.9706e-01, 2.9433e-03],
        [4.7275e-03, 9.9527e-01],
        [6.2977e-04, 9.9937e-01],
        [9.9583e-01, 4.1739e-03],
        [1.5903e-03, 9.9841e-01],
        [9.9934e-01, 6.5698e-04],
        [9.9417e-01, 5.8278e-03],
        [9.9710e-01, 2.9013e-03],
        [1.7240e-03, 9.9828e-01],
        [9.9314e-01, 6.8607e-03],
        [9.8784e-01, 1.2161e-02],
        [1.2313e-03, 9.9877e-01],
        [2.538

In [ ]:
df_val.shape

(92, 2)

In [ ]:
print(" Accuracy : ", accuracy(final_pred, targets_fwd))

predictions = np.argmax(final_pred , axis = 1)
confusion_matrix = pd.crosstab(predictions, targets_fwd)
confusion_matrix

 Accuracy :  tensor(0.9783)


col_0,0,1
row_0,,
0,51,1
1,1,39


In [ ]:
df_val['category'].value_counts()

business    52
tech        40
Name: category, dtype: int64

## Pretty cool results of 97.83% accuracy 